# Pomaluj mój świat
## Dekoratory

### Marcin Jaroszewski
### 22.III.2018, Python Level UP

![Logo kursu Python Level Up](https://raw.githubusercontent.com/daftcode/python_levelup_2018/master/logo.png)

![Plan zajęć](https://raw.githubusercontent.com/daftcode/python_levelup_2018/master/plan_zajec.png)

# Co to dekorator?

Dekorator to fragment kodu "opakowujący" inny fragment kodu.

Kod w "opakowaniu" może być wykonany względem kodu opakowanego:
* przed
* po
* zamiast

Dodatkowo kod "opakowujący" może dowolnie zmienić kod "opakowany".

Akt "opakowywania" dzieje się w momencie importowania modułu zwaierającego "opakowawywany" fragment kodu. Czyli w runtime. Ma to swoje konsekwencje, o których trzeba pamiętać (diabeł tkwi w szczegółach).

# Do czego są wykorzystywane dekoratory

* łatwego i czystego rozszerzania kodu
* tworzenia cache
* mechanizmy nieobecne bezpośrednio w python (frameworki)

# Co może być udekorowane

Funkcje - względnie łatwe.

Klasy - w zależności od sposobów użycia może być więcej lub mniej pracy. Moim zdaniem o rząd wielkości trudniejsze od dekoratorów funkcji.

# Czym można dekorować

Obiektami "callable" czyli:
* funkcjami
* klasami z metodą `__call__`

Jak zwykle funkcje sa łatwe do opanowania. Klasy ciągną za sobą bagaż związany z dziedziczeniem i instancjonowaniem, co sprawia, że są trudniejsze.

# Składnia

### Szkielet
```python
def nazwa_dekoratora(funkcja_opakowywana):
    #ciało dekoratora
    
@nazwa_dekoratora
def zwykla_funkcja():
    # ciało zwykłej funkcji
    
# użycie
zwykla_funkcja()
```

### Wyjaśnienie szkieletu
```python
def nazwa_dekoratora(funkcja_opakowywana):
    #ciało dekoratora
    
def zwykla_funkcja():
    # ciało zwykłej funkcji
    
# mniej więcej to znaczy @nazwa_dekoratora
zwykla_funkcja = nazwa_dekoratora(zwykla_funkcja)

# użycie
zwykla_funkcja()
```

# Wykonanie kodu zamiast udekorowanej funkcji

In [49]:
def censor(singer):
    print('Nie pozwalam wykonać: {}'.format(singer.__name__))

In [50]:
@censor
def play_music():
    print('Gra muzyka')

Nie pozwalam wykonać: play_music


In [51]:
play_music()

TypeError: 'NoneType' object is not callable

No i dramat - nie działa.

Ale cos jednak zadziałało - kod zawarty w `censor` został wykonany, ale nie wtedy kiedy chcieliśmy.
Kod został wykonany w momencie "opakowywania" (dekorowania), a nie  wmomencie wywołania funkcji `play_music`. W dodatku `play_music` stało sie w jakiś sposób `None`.

Czy ktoś wie **dlaczego**?

Żeby pójść dalej z dekoratorami musimy się cofnąć i przypomnieć sobie kilka konceptów i mechanizmów.

# Scope

Zmienne mogą być zdefniowane na różnych poziomach i mają wynikającą z tego "widoczność".

In [52]:
# Przykład na zmienne "globalne"
a = 1
b = 101

def add_something_to_a_and_b(something):
    a += something
    b += something
    print('add_something_to_a_and_b a:', a)
    print('add_something_to_a_and_b b:', b)

add_something_to_a_and_b(11)
print('a:', a)
print('b:', b)

UnboundLocalError: local variable 'a' referenced before assignment

In [53]:
# Przykład na zmienne "globalne" działający
a = 1
b = 101

def add_something_to_a_and_b(something):
    global a
    b = 101
    a += something
    b += something
    print('add_something_to_a_and_b a:', a, 'id a:', id(a))
    print('add_something_to_a_and_b b:', b, 'id b:', id(b))

add_something_to_a_and_b(11)
print('a:', a, 'id a:', id(a))
print('b:', b, 'id b:', id(b))

add_something_to_a_and_b a: 12 id a: 11062112
add_something_to_a_and_b b: 112 id b: 11065312
a: 12 id a: 11062112
b: 101 id b: 11064960


In [54]:
# Przykład na zmienne "globalne" z obiektami mutowalnymi
a = [1]

def add_something_to_a(something):
    a[0] += something
    print('add_something_to_a_and_b a:', a, 'id a:', id(a))

add_something_to_a(11)
print('a:', a, 'id a:', id(a))

add_something_to_a_and_b a: [12] id a: 140659921335176
a: [12] id a: 140659921335176


Używanie zmiennych globalnych jest uznawane za nieprofesjonalne :)
W większych projektach bardzo łatwo zrobić sobie i innym krzywdę modyfikując zawartość/stan zmiennych globalnych.

# Closure

In [55]:
def hey_hi_hello_factory(welcome_txt='hello'):
    def greet(name):
        print('{} {}!'.format(welcome_txt, name))
    return greet
        
say_hello = hey_hi_hello_factory('hello')
say_hi = hey_hi_hello_factory('hi')

say_hello('Janek')
say_hi('Magda')
say_hello('Maciek')

hello Janek!
hi Magda!
hello Maciek!


Mamy dostęp (read) do zmiennych zadeklarowanych w funkcji okalającej!

Mimo, że `say_hello` i `say_hi` są używane poza `hey_hi_hello_factory`!

In [56]:
def hey_hi_hello_factory(welcome_txt='hello'):
    counter = 0
    def greet(name):
        counter += 1
        print('{} {}!\tcounter: {}'.format(welcome_txt, name, counter))
    return greet
        
say_hello = hey_hi_hello_factory('hello')
say_hi = hey_hi_hello_factory('hi')

say_hello('Janek')
say_hi('Magda')
say_hello('Maciek')

UnboundLocalError: local variable 'counter' referenced before assignment

Dostęp (write) do zmiennej zadeklarowanej w funkcji okalającej nie działa :(

W python 2.7 skończylibyśmy w tym miejscu, bo nie było na ten problem rozwiązania eleganckiego.
Dostępne opcje to obiekty mutowalne i korzystanie z atrybutów funkcji.

Ale na szczęście jest już python w wersji 3.6 :)

In [57]:
def hey_hi_hello_factory(welcome_txt='hello'):
    counter = 0
    def greet(name):
        nonlocal counter
        counter += 1
        print('{} {}!\tcounter: {}'.format(welcome_txt, name, counter))
    return greet
        
say_hello = hey_hi_hello_factory('hello')
say_hi = hey_hi_hello_factory('hi')

say_hello('Janek')
say_hi('Magda')
say_hello('Maciek')

hello Janek!	counter: 1
hi Magda!	counter: 1
hello Maciek!	counter: 2


# Function as first class object

Zwykle podaje się przykład, że można funkcję do zmiennej przekazywać. Coś w stylu:

In [58]:
def hello(name):
    print('hello', name)

goodbye = hello

hello('Kuba')
goodbye('Arek')

hello Kuba
hello Arek


Ale w python można więcej!

Można ustawiać atrybuty na funkcjach!

In [59]:
def hello(name):
    hello.count += 1
    print('hello', name, 'count:', hello.count)

goodbye = hello

hello('Kuba')
goodbye('Arek')

AttributeError: 'function' object has no attribute 'count'

In [60]:
# najpierw atrybut trzeba zdefiniować :)
def hello(name):
    hello.count = 0
    hello.count += 1
    print('hello', name, 'count:', hello.count)

goodbye = hello

hello('Kuba')
goodbye('Arek')

hello Kuba count: 1
hello Arek count: 1


In [61]:
# najpierw atrybut trzeba zdefiniować, ale tylko raz :)
def hello(name):
    # nie za bardzo jest opcja wewnątrz funkcji
    hello.count += 1
    print('hello', name, 'count:', hello.count)
    
# ale można na zewnątrz
hello.count = 0

goodbye = hello

hello('Kuba')
goodbye('Arek')

hello Kuba count: 1
hello Arek count: 2


Wykorzystywanie atrybutów funkcji było powszechne w python 2.7 i wcześniejszych, ponieważ `nonlocal` jeszcze nie istniało. Wiele kodu jest przeportowane z 2.7 do 3.x i łatwo można się natknąć na wykorzystanie arybutów. W nowym kodzie zalecam używanie `nonlocal`.

# Wykonanie kodu zamiast udekorowanej funkcji, podejście nr 2

In [62]:
def censor(singer):
    def print_censor_msg():
        print('Nie pozwalam wykonać: {}'.format(singer.__name__))
    return print_censor_msg

In [63]:
@censor
def play_music():
    print('Gra muzyka')

In [64]:
play_music()

Nie pozwalam wykonać: play_music


In [65]:
play_music()
play_music()
play_music()

Nie pozwalam wykonać: play_music
Nie pozwalam wykonać: play_music
Nie pozwalam wykonać: play_music


Jeśli dekorujemy funkcję to dekorator powinien zwracać obiekt, który można "zawołać"/"wykonać" (callable)! 

# Wykonanie kodu przed udekorowaną funkcją

In [66]:
def przodownik(wolna_funkcja):
    def inner():
        print('Pierwszy!!1!')
        return wolna_funkcja()
    return inner

In [67]:
@przodownik
def play_music():
    print('Gra muzyka')

In [68]:
play_music()

Pierwszy!!1!
Gra muzyka


In [69]:
play_music()
play_music()
play_music()

Pierwszy!!1!
Gra muzyka
Pierwszy!!1!
Gra muzyka
Pierwszy!!1!
Gra muzyka


# Wykonanie kodu po udekorowanej funkcji

In [70]:
def bumelant(szybka_funkcja):
    def inner():
        result = szybka_funkcja()
        print('Zasada zachowania energii: zachowaj energię na później :)')
        return result
    return inner

In [71]:
@bumelant
def play_music():
    print('Gra muzyka')

In [72]:
play_music()

Gra muzyka
Zasada zachowania energii: zachowaj energię na później :)


# Utrata informacji - wraps

In [73]:
# wykorzystamy udekorowaną funkcję z poprzedniego przykładu
print(play_music.__name__)

inner


Dekorowanie funkcji zabiera nam część informacji o tym co miało być wykonywane. Możemy się bardzo zdziwić w przypadku jakiegoś błędu, ze nie rozumiemy czemu jakiś kawałek kodu się wykonał.

Ale jest na to rozwiązanie!

Kolejny dekorator - `wraps`.

In [74]:
from functools import wraps

def bumelant(szybka_funkcja):
    @wraps(szybka_funkcja)
    def inner():
        result = szybka_funkcja()
        print('Zasada zachowania energii: zachowaj energię na później :)')
        return result
    return inner

In [75]:
@bumelant
def play_music():
    print('Gra muzyka')

In [76]:
play_music()

Gra muzyka
Zasada zachowania energii: zachowaj energię na później :)


In [77]:
print(play_music.__name__)

play_music


# Więcej niż jedno zwierzę

Czy można funkcję/klasę udekorować więcej niż jednym dekoratorem? 

Można :)

### Składnia

```python
def dekorator_pierwszy(do_udekorowania):
    # ciało pierwszego dekoratora
    
def dekorator_drugi(do_udekorowania):
    # ciało drugirgo dekoratora
```

```python
@dekorator_pierwszy
@dekorator_drugi
def zwykla_funkcja():
    # ciało zwykłej funkcji
    
# to znaczy mniej więcej
zwykla_funkcja = dekorator_pierwszy(dekorator_drugi(zwykla_funkcja))
# kolejność wykonywania: od najbardziej wewnętrznego do najbardziej zewnętrznego
# czyli dekorator pierwszy od góry zostanie wykonany jako ostatni :)

# użycie
zwykla_funkcja()
```

Warto pamiętać o używaniu `wraps` - bardzo sie przydaje w przypadku "piętrowych" dekoratorów - ułatwia zrozumienie co się dzieje.

# Dekorowana funkcja z argumentami

Czasami bywa tak, że funkcja powinna przyjmować argumenty.

In [78]:
from functools import wraps

def bumelant(szybka_funkcja):
    @wraps(szybka_funkcja)
    def inner():
        result = szybka_funkcja()
        print('Zasada zachowania energii: zachowaj energię na później :)')
        return result
    return inner

In [79]:
@bumelant
def play(music):
    print('Gra: {}'.format(music))

In [80]:
play('jazz')

TypeError: inner() takes 0 positional arguments but 1 was given

In [81]:
from functools import wraps

def bumelant(szybka_funkcja):
    @wraps(szybka_funkcja)
    def inner(*args):
        result = szybka_funkcja(*args)
        print('Zasada zachowania energii: zachowaj energię na później :)')
        return result
    return inner

In [82]:
@bumelant
def play(music):
    print('Gra: {}'.format(music))

In [83]:
play('jazz')

Gra: jazz
Zasada zachowania energii: zachowaj energię na później :)


A co gdybyśmy chcieli następującą funkcję opakować?

In [84]:
@bumelant
def play(music='polskie regge'):
    print('Gra: {}'.format(music))

In [85]:
play('jazz')

Gra: jazz
Zasada zachowania energii: zachowaj energię na później :)


In [86]:
play()

Gra: polskie regge
Zasada zachowania energii: zachowaj energię na później :)


* Dlaczego zadziałało?
* A co się stanie gdy zmienimy sygnaturę funkcji opakowywanej na:  
```python
def play(*, music='polskie regge'):
```
* Czy można zrobić coś lepiej?


# Argumenty dekoratora

Z doświadczenia wiemy, że można przekazać argumenty bezpośrednio do dekoratora (`wraps`, `app.route`).

Wszystkie dotychczasowe nasze dekoratory przyjmowały za argument funkcję, którą mają opakować. Nie ma w nich miejsca na przekazanie argumentów w `@`.

In [87]:
@bumelant('co masz zrobić dziś zrób pojutrze, będziesz mieć 2 dni wolnego')
def play(music):
    print('Gra: {}'.format(music))

TypeError: 'str' object is not callable

Nie wiemy jeszcze jak napisać dekorator, który przyjmie argumenty. 

# Fundamental theorem of software engineering

## "We can solve any problem by introducing an extra level of indirection."

* https://en.wikipedia.org/wiki/Fundamental_theorem_of_software_engineering

# Argumenty dekoratora, podejście nr 2

Jeśli się poważniej zastanowimy nad poprzednim przykładem to możemy dojść do wniosku, że przekazanie argumentów do dekoratora działa mniej więcej w następujący sposób:
```python
dekorator(argumenty_dekoratora)(cos_do_udekorowania)(argumenty_czegos_do_udekorowania)
```

### Szkielet
```python
def dekorator(argumenty_dekoratora):
    # ta funkcja ma za zadanie przechwycić argumenty i zwrócić
    # "prawdziwy" dekorator
    def prawdziwy_dekorator(do_udekorowania):
        # Ta funkcja ma za zadanie przechwycić to co zostanie udekorowane.
        def wrapper(*args, **kwargs):
            # args i kwargs to argumenty funkcji dekorowanej
            return to_co_chcemy
        return wrapper
    return prawdziwy_dekorator
```

## Przykład 1

In [88]:
def bumelant(sentence):
    print('decorator called with sentence:', sentence)
    def real_decorator(to_be_decorated):
        def wrapper(*args, **kwargs):
            result = to_be_decorated(*args, **kwargs)
            print('Zasada zachowania energii: zachowaj energię na później :)')
            print('Sentencja dekoratora:', sentence)
            return result
        return wrapper
    return real_decorator

In [89]:
@bumelant('Co masz zrobić dziś zrób pojutrze, będziesz mieć 2 dni wolnego')
def play(music):
    print('Gra: {}'.format(music))

decorator called with sentence: Co masz zrobić dziś zrób pojutrze, będziesz mieć 2 dni wolnego


In [90]:
play('song of victory')

Gra: song of victory
Zasada zachowania energii: zachowaj energię na później :)
Sentencja dekoratora: Co masz zrobić dziś zrób pojutrze, będziesz mieć 2 dni wolnego


##  Przykład 2

In [91]:
def my_wraps(original):
    def outer_wrapper(to_be_decorated):
        def wrapper(*args, **kwargs):
            return to_be_decorated(*args, **kwargs)
        wrapper.__name__ = original.__name__
        return wrapper
    return outer_wrapper

In [92]:
def przodownik(wolna_funkcja):
    @my_wraps(wolna_funkcja)
    def inner(*args, **kwargs):
        print('Pierwszy!!1!')
        return wolna_funkcja(*args, **kwargs)
    return inner

In [93]:
@przodownik
def play(music):
    print('Gra: {}'.format(music))

In [94]:
play('disco')
print(play.__name__)

Pierwszy!!1!
Gra: disco
play


# Przykłady użytecznych dekoratorów 

* lista użytecznych dekoratorów: https://github.com/lord63/awesome-python-decorator

## app.route()

## Cross Site Request Forgery

Django ma ochronę przed tym zagrożeniem wbudowaną w formie dekoratora.  
"Goły" flask nie ma tego zabezpieczenia.

* opis różnych ataków i zabezpieczen w kontekście flask: http://flask.pocoo.org/docs/0.12/security/
* Rozszerzenie do flask do obsługi formularzy z zabezpieczeniem przed CSRF http://flask-wtf.readthedocs.io/en/stable/csrf.html

## classmethod

Służy do wykonywania metod nie na instancji danej klasy tylko na danej klasie. Używany zazwyczaj do tworzenia alternatywnych konstruktorów/inicjalizatorów - najczęsciej wariacje `from_string`.

## staticmethod

Służy do oznaczenia metody klasy jako statycznej. Takie metody nie przyjmują parametru `self` i są wywoływane na klasie a nie na instancji. Moim zdaniem takie metody lepiej przenieść do osobnych funkcji w module.

Artykuł o `classmethod` i `staticmethod`: http://stackabuse.com/pythons-classmethod-and-staticmethod-explained/

## functools.wraps

## property

Pythonowa odpowiedź na gettery i settery. Jest o wiele lepsza niż XX wieczne metody.

Pomocne linki:
* dokumentacja: https://docs.python.org/3/library/functions.html#property
* artykuł o tym jak używać: https://www.programiz.com/python-programming/property
* inny artykul o tym jak używać: http://stackabuse.com/python-properties/


## functools.lru_cache

Moim zdaniem najlepszy dekorator ze wszystkich jakie spotkałem, najbardziej użyteczny.
Dokumentacja: https://docs.python.org/3/library/functools.html#functools.lru_cache
Ale należy uważać na standardowe problemy związane z cache.

Czy ktoś z sali jest w stanie podać jakiś problem związany z cache?

# That's all folks!